# 04-Calculate Quadkeys

This notebook calculates the level 20 quadkeys based on the map-matched locations.

**Requirements:**

- Please run the `02-import-data.ipynb` notebook first and its dependencies.
- Recommended install: [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html). Enable using `jupyter nbextension enable --py widgetsnbextension --sys-prefix` for Jupyter Notebook and `jupyter labextension install @jupyter-widgets/jupyterlab-manager` for Jupyter Lab.

In [ ]:
from pyquadkey2 import quadkey
from db.api import EVedDb 
from tqdm.notebook import tqdm

Create the database API object.

In [ ]:
db = EVedDb()

Get all the vehicle trips into a list of tuples `(vehicle_id, trip_id)`.

In [ ]:
trips = db.query("select distinct vehicle_id, trip_id from signal;")

The function below extracts the unique locations from a vehicle trip.

In [ ]:
def get_trip_locations(vehicle_id, trip_id):
    sql = """
    select   match_latitude
    ,        match_longitude
    from     signal 
    where    vehicle_id = ? and trip_id = ?
    group by match_latitude, match_longitude
    order by time_stamp
    """

    locations = db.query(sql, (vehicle_id, trip_id))
    return locations

Use _zoom_ level 20 to discretize the quadkeys.

In [ ]:
level = 20

The query below updates quadkeys per vehicle and trip.

In [ ]:
sql = """
update signal set quadkey = ?
where  vehicle_id = ? and trip_id = ? and match_latitude = ? and match_longitude = ?
"""

Now we can update all quadkeys in the database.

**Note:** This code will take some time to run. On my 2019 i7 MacBook it took around 14 minutes.

In [ ]:
for trip in tqdm(trips):
    vehicle_id, trip_id = trip
    
    locations = get_trip_locations(vehicle_id, trip_id)
    
    updates = [(quadkey.from_geo(p, level).to_quadint() >> (64-2*level), vehicle_id, trip_id, p[0], p[1]) for p in locations]
    
    db.execute_sql(sql, updates, many=True)